In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


#### We can create a speech from the given input and store at some place

In [4]:
from pathlib import Path
import openai

# Define the path where the generated speech file will be saved
script_directory = Path.cwd()
speech_file_path = script_directory / "speech.mp3"

# Make a request to the OpenAI Audio API to generate speech
response = openai.audio.speech.create(
    model="tts-1",   # The latest text to speech model, optimized for speed.
    voice="alloy",   # The voice style for the generated speech
    input="The quick brown fox jumped over the lazy dog."   # The text to be converted to speech
)

# Save the generated speech to the specified file path
response.stream_to_file(speech_file_path)



# We can also convert into hd format

In [ ]:
from pathlib import Path
import openai

# Define the path where the generated speech file will be saved
script_directory = Path.cwd()
speech_file_path = script_directory / "speech.mp3"

# Make a request to the OpenAI Audio API to generate speech
response = openai.audio.speech.create(
    model="tts-1-hd",   # The latest text to speech model, optimized for quality.
    voice="alloy",   # The voice style for the generated speech
    input="The quick brown fox jumped over the lazy dog."   # The text to be converted to speech
)

# Save the generated speech to the specified file path
response.stream_to_file(speech_file_path)

#### Converting Speech into Text transcriptions means (that you have to send speech file and then you should b convert into any language )

In [ ]:
from openai import OpenAI
client = OpenAI()

audio_file = open("speech.mp3", "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file
)


### Converting Speech to text using translation (means that you have to only convert translate speech into english)

In [ ]:
from openai import OpenAI
client = OpenAI()

audio_file = open("speech.mp3", "rb")
transcript = client.audio.translations.create(
  model="whisper-1",
  file=audio_file
)


### First Function is taking audio file in hindi and converting it into english called translation function

### The Second Function is taking hindi audio file and convert into hindi text by defualt we can also pass another language instead of generating text in hindi

In [ ]:
audio_file = open("/content/different_language.mp3", "rb")
translated_transcript = client.audio.translations.create(
    model="whisper-1",
    response_format = "text",
    file=audio_file
)
print("Translated Transcript: ", translated_transcript)

original_transcript = client.audio.transcriptions.create(
    model="whisper-1",
    response_format="text",
    file=audio_file
)
print("Original Transcript: ", original_transcript)


#### So here is the problem how we can convert large files that are max of 25 mbs

#### So we have a solution for that we can convert large files into small segments using python library called pydub

Segmenting Audio for Efficient Processing

When dealing with lengthy audio files, it's often necessary to segment them for easier processing. PyDub, a flexible audio processing library in Python, is an excellent tool for this task.

Installing and Using PyDub

Start by installing PyDub:

In [1]:
!pip install pydub -q

In [5]:
from pydub import AudioSegment
# Load the audio file
song = AudioSegment.from_mp3("/content/NLP Roadmap 2024 Step-by-Step Guide Resources.mp3")

# PyDub handles time in milliseconds
five_minutes = 5 * 60 * 1000

# Extract the first 5 minutes
first_5_minutes = song[:five_minutes]

# Export the segment
first_5_minutes.export("split_speech.mp3", format="mp3")


#Using the Segmented Audio with Whisper

#With the segmented audio file, we can now efficiently utilize Whisper for transcription:

In [ ]:
audio_file = open("/content/split_speech.mp3", "rb")


Segmenting audio files is a practical approach to handling long recordings, making them more manageable for transcription or other audio processing tasks. PyDub's simplicity and efficiency make it an ideal choice for such operations.

Correcting Transcriptions with GPT-4
Enhancing Transcript Accuracy

Transcription errors are common, especially with unique terms or accents. In this section, we demonstrate how to use GPT-4 to correct transcription errors, focusing on specialized terminology related to Data Science.

The Process

Transcribe Audio: First, we transcribe the audio file using the Whisper API:

In [ ]:
def transcribe(audio_file):
     transcript = client.audio.translations.create(
         model="whisper-1",
         response_format="text",
         file=audio_file
     )
     return transcript

### Set Up the Correction Prompt: Prepare a system prompt instructing GPT-4 to correct spelling mistakes and ensure proper case usage for specialized terms.

In [6]:
 system_prompt = """You are given a video transcript with spelling mistakes...
 Rewrite transcript in the same format correcting spelling mistakes..."""


### Generate Corrected Transcript: Combine the transcription with GPT-4 to produce a corrected version:

In [ ]:
def generate_corrected_transcript(system_prompt, audio_file):
     text = transcribe(audio_file)
     response = client.chat.completions.create(
         model="gpt-4",
         temperature=0,
         messages=[
             {"role": "system", "content": system_prompt},
             {"role": "user", "content": text}
         ]
     )
     return response.choices[0].message.content

 audio_file = open("/content/split_speech.mp3", "rb")
 corrected_text = generate_corrected_transcript(system_prompt, audio_file)
 print(corrected_text)